In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from sklearn.neighbors import NearestNeighbors

In [2]:
# Conexion con Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cambio de directorio de Trabajo
import os
import sys
path = '/content/drive/MyDrive/Mod2/ANALITICA3'
os.chdir(path)
sys.path.append(path)

In [4]:
import sqlite3 as sql
import pandas as pd
import numpy as np

conn=sql.connect('/content/drive/MyDrive/Mod2/ANALITICA3/data/db_movies')
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [5]:
### para verificar las tablas que hay disponibles
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('peliculas_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('movie_genres',),
 ('df_genres',)]

In [6]:
# Cargas la tabla que ya contiene los datos procesados
tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

tabla_completa

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703,1995
1,1,3,Grumpier Old Men,Comedy|Romance,4.0,964981247,1995
2,1,6,Heat,Action|Crime|Thriller,4.0,964982224,1995
3,1,47,Seven (a.k.a. Se7en),Mystery|Thriller,5.0,964983815,1995
4,1,50,"Usual Suspects, The",Crime|Mystery|Thriller,5.0,964982931,1995
...,...,...,...,...,...,...,...
25973,607,3785,Scary Movie,Comedy|Horror,2.0,964743188,2000
25974,607,3793,X-Men,Action|Adventure|Sci-Fi,3.0,964743155,2000
25975,607,3994,Unbreakable,Drama|Sci-Fi,3.0,997847202,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action|Drama|Romance,2.0,997847173,2000


In [7]:
tabla_completa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25978 entries, 0 to 25977
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          25978 non-null  int64  
 1   movie_id         25978 non-null  int64  
 2   movie_title      25978 non-null  object 
 3   movie_genres     25978 non-null  object 
 4   movie_rating     25978 non-null  float64
 5   movie_timestamp  25978 non-null  int64  
 6   movie_year       25978 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 1.4+ MB


In [8]:
# Separar los géneros por el carácter '|' y expandirlos en múltiples filas, se hace de esta manera ya que sqlite no tiene esa funcion
df_genres = tabla_completa.assign(movie_genres=tabla_completa['movie_genres'].str.split('|')).explode('movie_genres')

# Renombrar para claridad
df_genres = df_genres.rename(columns={'movie_genres': 'genre'})

df_genres

,user_id,movie_id,movie_title,genre,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure,4.0,964982703,1995
0,1,1,Toy Story,Animation,4.0,964982703,1995
0,1,1,Toy Story,Children,4.0,964982703,1995
0,1,1,Toy Story,Comedy,4.0,964982703,1995
0,1,1,Toy Story,Fantasy,4.0,964982703,1995
...,...,...,...,...,...,...,...
25975,607,3994,Unbreakable,Sci-Fi,3.0,997847202,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action,2.0,997847173,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Drama,2.0,997847173,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Romance,2.0,997847173,2000


In [9]:
#Crear nueva tabla en la base de datos
df_genres = df_genres.to_sql("df_genres", conn, if_exists='replace', index=False)

In [10]:
#verificar la nueva tabla
df_genres = pd.read_sql("SELECT * FROM df_genres", conn)

In [11]:
tabla_generos = pd.read_sql_query('''
SELECT
movie_title,
genre

FROM df_genres
''', conn)
tabla_generos

,movie_title,genre
0,Toy Story,Adventure
1,Toy Story,Animation
2,Toy Story,Children
3,Toy Story,Comedy
4,Toy Story,Fantasy
...,...,...
77446,Unbreakable,Sci-Fi
77447,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action
77448,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Drama
77449,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Romance


In [14]:
tabla_generos.nunique()

,0
movie_title,450
genre,19


In [12]:
df_genres['movie_title'].value_counts()

,count
movie_title,
Forrest Gump,868
Pulp Fiction,812
Shrek,726
"Lion King, The",714
Toy Story,685
...,...
Zoolander,30
My Cousin Vinny,29
Anchorman: The Legend of Ron Burgundy,29


In [13]:
tabla_generos['num_ratings'].info()

KeyError: 'num_ratings'

In [ ]:
df_generos = pd.read_sql('''
CREATE TABLE movie_genres AS
SELECT movie_title, genres,
COUNT(*) AS num_ratings,
AVG(movie_rating) AS avg_rating

FROM full_ratings
GROUP BY movie_title. genres''', conn)

In [ ]:
df_genres